<a href="https://colab.research.google.com/github/bharathbolla/The-LLM-Cookbook-Practical-Recipes-for-Fine-Tuning-Optimization-and-Deployment/blob/main/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recipe-1. Multi-Class Classification (DistilBERT)

In [ ]:
pip install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=37767c689f66f4451841e024dd4e2a97a005c896f0cbbfe1500d5d9f0e9dd29b
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
to

In [ ]:
pip install transformers datasets evaluate accelerate torch seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from huggingface_hub import HfApi
from huggingface_hub import login

api = HfApi()
whoami = api.whoami(token="hf_xxxxxxxxxxxxxxxx")
print(whoami)
login("hf_xxxxxxxxxxxxxxxxxxxxxx")

{'type': 'user', 'id': '65feba1b57cc48d9d30d11cf', 'name': 'kalpasubbaiah', 'fullname': 'Kalpa Subbaiah', 'email': 'kalpa.subbaiah@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/319094e0eb55ce89334d7bd3685ceeb0.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'hugging_face_token_read', 'role': 'read', 'createdAt': '2025-04-22T09:03:46.223Z'}}}


In [ ]:
import transformers
print("Transformers version:", transformers.__version__)
print("TrainingArguments source:", transformers.TrainingArguments.__module__)

Transformers version: 4.51.1
TrainingArguments source: transformers.training_args


In [ ]:
# --- Recipe: Multi-Class Classification (AG News / DistilBERT) ---
# Goal: Full fine-tuning of DistilBERT for multi-class text classification.
# Dataset: AG News (4 classes: World, Sports, Business, Sci/Tech)
# Libraries: transformers, datasets, evaluate, torch
# Note: Ensure libraries are installed: pip install transformers datasets evaluate accelerate torch


import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding

)

2025-05-31 16:37:40.788298: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748709461.030554      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748709461.101277      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:

# --- Configuration ---
MODEL_CHECKPOINT = "distilbert-base-uncased" # Baseline model
DATASET_NAME = "ag_news" # Multi-class news topic classification
NUM_EPOCHS = 1 # Use 1 epoch for quick demo, 2-3 might yield better results
BATCH_SIZE = 16 # Adjust based on GPU memory
LEARNING_RATE = 2e-5
OUTPUT_DIR = "./fine_tune_distilbert_agnews_output" # Directory for this model

# --- 1. Load Data ---
print(f"Loading dataset: {DATASET_NAME}")
try:
    raw_datasets = load_dataset(DATASET_NAME)
    # AG News has 'train' and 'test' splits. Create a validation split from train.
    train_val_split = raw_datasets["train"].train_test_split(test_size=0.1, seed=42)
    raw_datasets["train"] = train_val_split["train"]
    raw_datasets["validation"] = train_val_split["test"]
    print("Dataset loaded and split:")
    print(raw_datasets)
    # Get label mapping
    label_feature = raw_datasets["train"].features["label"]
    num_labels = label_feature.num_classes
    id_to_label = {i: label for i, label in enumerate(label_feature.names)}
    print(f"Number of labels: {num_labels}")
    print(f"Label mapping: {id_to_label}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

# --- 2. Load Tokenizer ---
print(f"\nLoading tokenizer for checkpoint: {MODEL_CHECKPOINT}")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    exit()

# --- 3. Preprocess Data ---
def tokenize_function(examples):
    # Tokenize text (column name is 'text' in ag_news)
    return tokenizer(examples["text"], truncation=True, padding=False)

print("\nTokenizing dataset...")
try:
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    # Rename 'label' to 'labels' for the Trainer
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    # Remove original text column
    tokenized_datasets = tokenized_datasets.remove_columns(["text"])
    tokenized_datasets.set_format("torch")
    print("Tokenization complete. Sample:")
    print(tokenized_datasets["train"][0])
except Exception as e:
    print(f"Error during tokenization: {e}")
    exit()

# --- 4. Data Collator ---
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- 5. Load Model ---
print(f"\nLoading model for sequence classification: {MODEL_CHECKPOINT}")
try:
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT,
        num_labels=num_labels
        # Optionally pass id_to_label, label_to_id if needed later
    )
    print(f"Model loaded. Initial device: {model.device}")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# --- 6. Evaluation Metric ---
print("\nLoading evaluation metric: accuracy")
try:
    accuracy_metric = evaluate.load("accuracy")
    # Could also load F1: f1_metric = evaluate.load("f1")
except Exception as e:
    print(f"Error loading metric: {e}")
    exit()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    # Example for adding F1 (requires label count for average)
    # f1 = f1_metric.compute(predictions=predictions, references=labels, average='macro') # or 'weighted'
    # return {"accuracy": acc["accuracy"], "f1": f1["f1"]}
    return acc # Just return accuracy for simplicity here

# --- 7. Training Arguments ---
print("\nDefining Training Arguments...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    report_to="none"
)
print(f"Training on device: {training_args.device}")

# --- 8. Initialize Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- 9. Train ---
print("\nStarting training...")
try:
    train_result = trainer.train()
    print("Training finished.")
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    trainer.save_model(OUTPUT_DIR) # Save final best model
    print(f"Model and training state saved to {OUTPUT_DIR}")
except Exception as e:
    print(f"Error during training: {e}")
    exit()

# --- 10. Evaluate on Test Set ---
print("\nEvaluating on the test set...")
try:
    test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
    trainer.log_metrics("test", test_results)
    trainer.save_metrics("test", test_results)
    print("Test Set Evaluation Results:")
    print(test_results)
except Exception as e:
    print(f"Error during test set evaluation: {e}")

# --- End of Recipe ---

Loading dataset: ag_news
Dataset loaded and split:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 108000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 12000
    })
})
Number of labels: 4
Label mapping: {0: 'World', 1: 'Sports', 2: 'Business', 3: 'Sci/Tech'}

Loading tokenizer for checkpoint: distilbert-base-uncased

Tokenizing dataset...


Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

Tokenization complete. Sample:
{'labels': tensor(0), 'input_ids': tensor([  101, 13905,  1998,  4963,  1999,  2235,  2845,  2237,  2044,  6859,
         2022, 14540,  2319,  1010,  3607,  1006, 26665,  1007,  1011,  1996,
         4288,  1997,  2062,  2084, 13710,  2336,  1010,  3008,  1998,  5089,
         2076,  1996,  6703,  2203,  2000,  1037,  5187,  1011,  3178,  2082,
         6859,  2187,  4510,  1037,  2155, 22154,  1999,  1996,  2235,  2845,
         2237,  1997,  2022, 14540,  2319,  1012,   102]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1])}

Loading model for sequence classification: distilbert-base-uncased


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded. Initial device: cpu

Loading evaluation metric: accuracy

Defining Training Arguments...
Training on device: cuda:0


/tmp/ipykernel_31/3240763051.py:131: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Starting training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.204700,0.181924,0.942417


Training finished.
***** train metrics *****
  epoch                    =        1.0
  total_flos               =  2527117GF
  train_loss               =     0.2297
  train_runtime            = 0:10:14.89
  train_samples_per_second =    175.638
  train_steps_per_second   =     10.977
Model and training state saved to ./fine_tune_distilbert_agnews_output

Evaluating on the test set...


***** test metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9375
  eval_loss               =     0.1865
  eval_runtime            = 0:00:12.45
  eval_samples_per_second =    610.029
  eval_steps_per_second   =     38.127
Test Set Evaluation Results:
{'eval_loss': 0.1865331530570984, 'eval_accuracy': 0.9375, 'eval_runtime': 12.4584, 'eval_samples_per_second': 610.029, 'eval_steps_per_second': 38.127, 'epoch': 1.0}


## Recipe-3. Multi-Class Classification (Gemma-2B)

In [ ]:
# --- Recipe: Multi-Class Classification (AG News / Gemma-2B) ---
# Goal: Full fine-tuning of Gemma-2B (a decoder-only SLM) for multi-class text classification.
# Dataset: AG News (4 classes: World, Sports, Business, Sci/Tech)
# Libraries: transformers, datasets, evaluate, torch, accelerate
# Note: Ensure libraries are installed: pip install transformers datasets evaluate accelerate torch sentencepiece
#       Fine-tuning Gemma-2B requires significant GPU memory (>16GB VRAM recommended, may need adjustments).

import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# --- Configuration ---
MODEL_CHECKPOINT = "google/gemma-2b" # Using Gemma 2B base model
DATASET_NAME = "ag_news"
NUM_EPOCHS = 1 # Use 1 epoch for quick demo
# Gemma-2B is larger, requires smaller batch size or gradient accumulation
BATCH_SIZE = 4 # Reduced batch size
GRADIENT_ACCUMULATION_STEPS = 4 # Simulate batch size of 4*4=16
LEARNING_RATE = 1e-5 # Often need smaller LR for larger models
OUTPUT_DIR = "./fine_tune_gemma_agnews_output" # Directory for this model

# Check GPU availability and VRAM
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    # Simple VRAM check (may not be perfectly accurate)
    # t = torch.cuda.get_device_properties(0).total_memory
    # r = torch.cuda.memory_reserved(0)
    # a = torch.cuda.memory_allocated(0)
    # free = t - (r + a) # free Gpu memory
    # print(f"Total VRAM: {t / 1024**3:.2f} GB")
    # print(f"Approx Free VRAM: {free / 1024**3:.2f} GB")
    # if free / 1024**3 < 14: # Rough estimate for Gemma 2B FT
    #     print("Warning: Low VRAM detected, fine-tuning Gemma-2B might fail.")
    #     print("Consider using QLoRA (Chapter 8) or a smaller model.")
else:
    print("Warning: No GPU detected. Fine-tuning Gemma-2B on CPU will be extremely slow.")


2025-05-31 09:35:00.946447: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748684101.167890      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748684101.236500      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU detected: Tesla T4


In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch

In [ ]:
# --- 1. Load Data ---
print(f"Loading dataset: {DATASET_NAME}")
try:
    raw_datasets = load_dataset(DATASET_NAME)
    train_val_split = raw_datasets["train"].train_test_split(test_size=0.1, seed=42)
    raw_datasets["train"] = train_val_split["train"]
    raw_datasets["validation"] = train_val_split["test"]
    #print("Dataset loaded and split:")
    #print(raw_datasets[:2])
    label_feature = raw_datasets["train"].features["label"]
    num_labels = label_feature.num_classes
    id_to_label = {i: label for i, label in enumerate(label_feature.names)}
    print(f"Number of labels: {num_labels}")
    print(f"Label mapping: {id_to_label}")
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()

# --- 2. Load Tokenizer ---
print(f"\nLoading tokenizer for checkpoint: {MODEL_CHECKPOINT}")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    # Gemma tokenizer might use <pad> ID 0, but explicitly setting can avoid issues
    if tokenizer.pad_token is None:
        print("Setting pad_token to eos_token")
        tokenizer.pad_token = tokenizer.eos_token
        # Important: Resize model embeddings if adding new special tokens AFTER model load
        # model.resize_token_embeddings(len(tokenizer))
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    exit()

# --- 3. Preprocess Data ---
def tokenize_function(examples):
    # Max length depends on model, Gemma typically supports 2048 or more, but keep shorter for classification
    return tokenizer(examples["text"], truncation=True, padding=False, max_length=512)

print("\nTokenizing dataset...")
try:
    tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
    tokenized_datasets = tokenized_datasets.remove_columns(["text"])
    tokenized_datasets.set_format("torch")
    print("Tokenization complete.")
except Exception as e:
    print(f"Error during tokenization: {e}")
    exit()

# --- 4. Data Collator ---
# Use standard padding collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# --- 5. Load Model ---
print(f"\nLoading model for sequence classification: {MODEL_CHECKPOINT}")
print("This may take a while and require significant RAM/VRAM...")
try:
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_CHECKPOINT,
        num_labels=num_labels,
        # Add pad_token_id if you explicitly set tokenizer.pad_token = tokenizer.eos_token
        # pad_token_id=tokenizer.pad_token_id,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else torch.float32 # Use bfloat16 if possible
    )
    # If PAD token was added/changed AFTER loading tokenizer but BEFORE model, resize needed:
    # model.resize_token_embeddings(len(tokenizer))
    # Explicitly set pad_token_id in model config if it wasn't set correctly
    if model.config.pad_token_id is None:
         model.config.pad_token_id = tokenizer.pad_token_id
         print(f"Set model.config.pad_token_id to: {model.config.pad_token_id}")

    print(f"Model loaded. Initial device: {model.device}") # Trainer will move it
except Exception as e:
    print(f"Error loading model: {e}")
    print("Check VRAM, RAM, and potential Hugging Face Hub connectivity issues.")
    exit()

# --- 6. Evaluation Metric ---
print("\nLoading evaluation metric: accuracy")
try:
    accuracy_metric = evaluate.load("accuracy")
except Exception as e:
    print(f"Error loading metric: {e}")
    exit()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# --- 7. Training Arguments ---
print("\nDefining Training Arguments...")
# Check for bfloat16 support for mixed precision
bf16_supported = torch.cuda.is_available() and torch.cuda.is_bf16_supported()

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE * 2, # Can often use larger eval batch size
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50, # Log more frequently with accumulation
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available() and not bf16_supported, # Use fp16 if bf16 not supported
    bf16=bf16_supported, # Use bf16 if supported (preferred for newer GPUs)
    gradient_checkpointing=True, # Saves memory at cost of slower training
    push_to_hub=False,
    report_to="none"
)
print(f"Training on device: {training_args.device}")
print(f"Using Gradient Accumulation: {GRADIENT_ACCUMULATION_STEPS} steps")
print(f"Effective Batch Size: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}")
print(f"Mixed precision (fp16): {training_args.fp16}")
print(f"Mixed precision (bf16): {training_args.bf16}")

# --- 8. Initialize Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- 9. Train ---
print("\nStarting training (Gemma-2B)...")
try:
    train_result = trainer.train()
    print("Training finished.")
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    trainer.save_model(OUTPUT_DIR) # Save final best model
    print(f"Model and training state saved to {OUTPUT_DIR}")
except Exception as e:
    print(f"Error during training: {e}")
    print("This might be an Out-of-Memory (OOM) error. Try reducing batch size, enabling gradient checkpointing, or using LoRA/QLoRA (Chapter 8).")
    exit()

# --- 10. Evaluate on Test Set ---
print("\nEvaluating on the test set...")
try:
    test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
    trainer.log_metrics("test", test_results)
    trainer.save_metrics("test", test_results)
    print("Test Set Evaluation Results:")
    print(test_results)
except Exception as e:
    print(f"Error during test set evaluation: {e}")

# --- End of Recipe ---


Loading dataset: ag_news


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

...


Starting training (Gemma-2B)...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Error during training: CUDA out of memory. Tried to allocate 1000.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 338.12 MiB is free. Process 5098 has 14.41 GiB memory in use. Of the allocated memory 14.05 GiB is allocated by PyTorch, and 177.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
This might be an Out-of-Memory (OOM) error. Try reducing batch size, enabling gradient checkpointing, or using LoRA/QLoRA (Chapter 8).

Evaluating on the test set...


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,2.309317,0.270395


***** test metrics *****
  eval_accuracy = 0.2704
  eval_loss     = 2.3093
Test Set Evaluation Results:
{'eval_loss': 2.3093173503875732, 'eval_accuracy': 0.27039473684210524}


## Recipe-4: Generative Adaptation (Causal LM Fine-Tuning)

In [ ]:
# --- Recipe: Causal LM Fine-Tuning on Wikitext-2 ---
# Goal: Fine-tune GPT-2 on Wikitext-2 with tokenization and block grouping

import torch
import math
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline,
)

# --- Configuration ---
MODEL_CHECKPOINT = "distilgpt2"
DATASET_NAME = "wikitext"
DATASET_CONFIG = "wikitext-2-raw-v1"
NUM_EPOCHS = 1
BATCH_SIZE = 4
LEARNING_RATE = 2e-5
BLOCK_SIZE = 128
OUTPUT_DIR = "./fine_tuned_gpt2_wikitext"

# --- 1. Load Dataset Subsets ---
print("Loading Wikitext dataset...")
train_dataset = load_dataset(DATASET_NAME, DATASET_CONFIG, split="train[:5000]")
val_dataset = load_dataset(DATASET_NAME, DATASET_CONFIG, split="validation[:500]")

# --- 2. Load Tokenizer ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(f"Set PAD token to EOS token: {tokenizer.pad_token}")

# --- 3. Tokenization ---
def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# --- 4. Group Tokens into Blocks ---
def group_texts(examples):
    # Concatenate all examples for each key
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])

    # Truncate to a multiple of block_size
    if total_length >= BLOCK_SIZE:
        total_length = (total_length // BLOCK_SIZE) * BLOCK_SIZE

    # Split into blocks
    result = {
        k: [concatenated[k][i : i + BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
        for k in concatenated.keys()
    }

    return result

lm_train_dataset = tokenized_train.map(group_texts, batched=True)
lm_val_dataset = tokenized_val.map(group_texts, batched=True)

# --- 5. Load Model ---
model = AutoModelForCausalLM.from_pretrained(MODEL_CHECKPOINT)

# --- 6. Data Collator ---
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# --- 7. Training Arguments ---
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=50,
    push_to_hub=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=torch.cuda.is_available(),
    report_to="none",
)

# --- 8. Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_train_dataset,
    eval_dataset=lm_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# --- 9. Train ---
print("\nTraining...")
trainer.train()
trainer.save_model(OUTPUT_DIR)

# --- 10. Evaluate (Perplexity) ---
print("\nEvaluating model...")
eval_results = trainer.evaluate()
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity:.2f}")

# --- 11. Generate Text (Optional) ---
print("\nGenerating text...")
generator = pipeline("text-generation", model=OUTPUT_DIR, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)
prompt = "Artificial intelligence is"
output = generator(prompt, max_new_tokens=50, num_return_sequences=1)[0]["generated_text"]
print(f"Prompt: {prompt}\nGenerated: {output}")


Loading Wikitext dataset...
Set PAD token to EOS token: <|endoftext|>


/tmp/ipykernel_31/3786389367.py:90: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Training...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,1.967800,3.797837


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['lm_head.weight'].



Evaluating model...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Device set to use cuda:0


Perplexity: 44.60

Generating text...
Prompt: Artificial intelligence is
Generated: Artificial intelligence is an area of scientific research that often has its limitations due to the complexity and complexity of the data. The underlying cause and source of these anomalies is the "stalker effect". In a 2003 article entitled A New Approach to Complex Intelligence, Stephen Hawking and


## Recipe 2.Token Classification Specialization (NER)



In [ ]:
# --- Recipe: Token Classification Specialization (NER) ---
# Goal: Full fine-tuning of a pre-trained model for Token Classification (Named Entity Recognition).
# Libraries: transformers, datasets, evaluate, seqeval, torch
# Note: Ensure libraries are installed: pip install transformers datasets evaluate seqeval accelerate torch
#       Uses DistilBERT and CoNLL-2003 dataset. Requires careful label alignment.

import torch
import evaluate
import numpy as np
from datasets import load_dataset, ClassLabel
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification # Specific data collator for token tasks
)

# --- Configuration ---
MODEL_CHECKPOINT = "distilbert-base-uncased"
DATASET_NAME = "conll2003" # Common NER dataset
NUM_EPOCHS = 1 # Use 1 epoch for quick demo
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
OUTPUT_DIR = "./fine_tune_ner_output"
# Use a task name prefix for potential model hub uploads (optional)
TASK_NAME = "ner"
LABEL_ALL_TOKENS = True # Whether to align labels for all sub-tokens or just the first

# --- 1. Load Data & Labels ---
print(f"Loading dataset: {DATASET_NAME}")
try:
    raw_datasets = load_dataset(DATASET_NAME)
    print("Dataset loaded:")
    print(raw_datasets)

    # Extract label list and mappings from dataset features
    ner_features = raw_datasets["train"].features["ner_tags"]
    label_list = ner_features.feature.names
    label_to_id = {label: i for i, label in enumerate(label_list)}
    id_to_label = {i: label for i, label in enumerate(label_list)}
    num_labels = len(label_list)

    print(f"\nLabels: {label_list}")
    print(f"Number of labels: {num_labels}")
    print(f"\nExample tokens: {raw_datasets['train'][0]['tokens']}")
    print(f"Example NER tags (IDs): {raw_datasets['train'][0]['ner_tags']}")
    print(f"Example NER tags (Labels): {[id_to_label[tag_id] for tag_id in raw_datasets['train'][0]['ner_tags']]}")

except Exception as e:
    print(f"Error loading dataset or extracting labels: {e}")
    exit()

# --- 2. Load Tokenizer ---
print(f"\nLoading tokenizer for checkpoint: {MODEL_CHECKPOINT}")
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
    # Check if the tokenizer is fast (required for offset mapping used in label alignment)
    if not tokenizer.is_fast:
        raise TypeError(
            "This recipe requires a fast tokenizer for offset mapping. "
            f"Try using {MODEL_CHECKPOINT} without '-base-uncased' or choose another fast tokenizer."
        )
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    exit()

# --- 3. Preprocess Data (Tokenize and Align Labels) ---
# Function to tokenize inputs and align labels with sub-tokens
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True, # Important: Input is already split into words
        padding=False # Padding handled by collator
    )

    labels = []
    for i, label in enumerate(examples[f"{TASK_NAME}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i) # Map tokens to original words
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For subsequent tokens in a multi-token word, set label based on LABEL_ALL_TOKENS flag
            else:
                label_ids.append(label[word_idx] if LABEL_ALL_TOKENS else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

print("\nTokenizing dataset and aligning labels...")
try:
    tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)
    # Remove original columns no longer needed
    tokenized_datasets = tokenized_datasets.remove_columns(raw_datasets["train"].column_names)
    print("Tokenization and alignment complete. Sample:")
    sample = tokenized_datasets["train"][0]
    print(f"Input IDs: {sample['input_ids']}")
    print(f"Labels: {sample['labels']}")
    # Decode to see alignment (special tokens have label -100)
    for token_id, label_id in zip(sample['input_ids'], sample['labels']):
         if label_id != -100:
              print(f"{tokenizer.decode([token_id])}: {id_to_label.get(label_id, 'N/A')}")
         else:
              print(f"{tokenizer.decode([token_id])}: IGNORED")

except Exception as e:
    print(f"Error during tokenization/alignment: {e}")
    exit()

# --- 4. Data Collator ---
# Handles dynamic padding for token classification tasks
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# --- 5. Load Model ---
print(f"\nLoading model for Token Classification: {MODEL_CHECKPOINT}")
try:
    model = AutoModelForTokenClassification.from_pretrained(
        MODEL_CHECKPOINT,
        num_labels=num_labels,
        id2label=id_to_label, # Pass mappings for better inference/display
        label2id=label_to_id
    )
    print(f"Model loaded. Initial device: {model.device}")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# --- 6. Evaluation Metric (Seqeval) ---
print("\nLoading evaluation metric: seqeval (for NER)")
try:
    seqeval_metric = evaluate.load("seqeval")
except Exception as e:
    print(f"Error loading metric: {e}")
    exit()

# Function to compute metrics using seqeval
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2) # Get most likely label ID for each token

    # Remove ignored index (-100) and convert IDs to label strings
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval_metric.compute(predictions=true_predictions, references=true_labels)
    # Return main metrics, can add breakdown per class if needed
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# --- 7. Training Arguments ---
print("\nDefining Training Arguments...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="f1", # Use F1 score for NER typically
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    report_to="none"
)
print(f"Training on device: {training_args.device}")

# --- 8. Initialize Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- 9. Train ---
print("\nStarting training...")
try:
    train_result = trainer.train()
    print("Training finished.")
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    trainer.save_model(OUTPUT_DIR)
    print(f"Model and training state saved to {OUTPUT_DIR}")
except Exception as e:
    print(f"Error during training: {e}")
    exit()

# --- 10. Evaluate on Test Set ---
print("\nEvaluating on the test set...")
try:
    test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
    trainer.log_metrics("test", test_results)
    trainer.save_metrics("test", test_results)
    print("Test Set Evaluation Results:")
    print(test_results)
except Exception as e:
    print(f"Error during test set evaluation: {e}")

# --- End of Recipe ---


2025-05-31 09:14:17.149887: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748682857.666576      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748682857.804361      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading dataset: conll2003


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


...

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.114600,0.098826,0.864806,0.883768,0.874184,0.973088


Training finished.
***** train metrics *****
  epoch                    =        1.0
  total_flos               =   175116GF
  train_loss               =     0.2506
  train_runtime            = 0:01:06.52
  train_samples_per_second =    211.054
  train_steps_per_second   =      6.599
Model and training state saved to ./fine_tune_ner_output

Evaluating on the test set...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


***** test metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9669
  eval_f1                 =     0.8476
  eval_loss               =     0.1253
  eval_precision          =     0.8413
  eval_recall             =     0.8539
  eval_runtime            = 0:00:05.62
  eval_samples_per_second =     613.85
  eval_steps_per_second   =     19.199
Test Set Evaluation Results:
{'eval_loss': 0.12528881430625916, 'eval_precision': 0.8413236669784846, 'eval_recall': 0.8539045810586281, 'eval_f1': 0.847567440216751, 'eval_accuracy': 0.9669221738830192, 'eval_runtime': 5.6252, 'eval_samples_per_second': 613.85, 'eval_steps_per_second': 19.199, 'epoch': 1.0}


## Recipe-5: Logging with Weights & Biases

In [ ]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# --- Configuration (Same as Classification Recipe) ---
MODEL_CHECKPOINT = "distilbert-base-uncased"
DATASET_NAME = "imdb"
NUM_EPOCHS = 1
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
OUTPUT_DIR = "./fine_tune_classifier_wandb_output" # Use a different output dir

# --- Steps 1-6 (Load Data, Tokenizer, Preprocess, Collator, Model, Metrics) ---
# These steps are identical to the 'ch6_recipe_classification_ft' recipe.
# For brevity, assume these steps have been executed and we have:
# - tokenized_datasets (containing 'train', 'validation', 'test')
# - tokenizer
# - model
# - data_collator
# - compute_metrics function
# --- Start copy-paste block from ch6_recipe_classification_ft (if running standalone) ---
print(f"Loading dataset: {DATASET_NAME}")
raw_datasets = load_dataset(DATASET_NAME)
train_val_split = raw_datasets["train"].train_test_split(test_size=0.1, seed=42)
raw_datasets["train"] = train_val_split["train"]
raw_datasets["validation"] = train_val_split["test"]
print(f"Loading tokenizer for checkpoint: {MODEL_CHECKPOINT}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=False)
print("Tokenizing dataset...")
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
print(f"Loading model for sequence classification: {MODEL_CHECKPOINT}")
num_labels = raw_datasets["train"].features["label"].num_classes
model = AutoModelForSequenceClassification.from_pretrained(MODEL_CHECKPOINT, num_labels=num_labels)
print("Loading evaluation metric: accuracy")
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)
# --- End copy-paste block ---


# --- 7. Training Arguments (Modified for W&B) ---
print("\nDefining Training Arguments with W&B Reporting...")
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50, # Log more frequently to see updates in W&B
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=torch.cuda.is_available(),
    push_to_hub=False,
    # *** Key Change: Enable W&B reporting ***
    report_to="wandb",
    # Optional: Name your W&B run
    run_name=f"{MODEL_CHECKPOINT}-ft-{DATASET_NAME}-demo"
)
print(f"Training on device: {training_args.device}")
print(f"Reporting to W&B project: {WANDB_PROJECT_NAME}")

# --- 8. Initialize Trainer (Same as before) ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# --- 9. Train (W&B logging happens automatically) ---
print("\nStarting training (logging to W&B)...")
# You should see W&B links printed in your console output.
try:
    train_result = trainer.train()
    print("Training finished.")
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()
    trainer.save_model(OUTPUT_DIR)
    print(f"Model and training state saved to {OUTPUT_DIR}")
except Exception as e:
    print(f"Error during training: {e}")
    # Ensure W&B run is finished even if there's an error
    import wandb
    wandb.finish()
    exit()

# --- 10. Evaluate on Test Set (Also logged to W&B) ---
print("\nEvaluating on the test set (logging to W&B)...")
try:
    test_results = trainer.evaluate(eval_dataset=tokenized_datasets["test"])
    trainer.log_metrics("test", test_results)
    trainer.save_metrics("test", test_results)
    print("Test Set Evaluation Results:")
    print(test_results)
except Exception as e:
    print(f"Error during test set evaluation: {e}")

# --- 11. Finish W&B Run ---
# Important to properly close the W&B run
import wandb
wandb.finish()
print("\nW&B run finished.")

# --- End of Recipe ---


Loading dataset: imdb
Loading tokenizer for checkpoint: distilbert-base-uncased
Tokenizing dataset...


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loading model for sequence classification: distilbert-base-uncased
Loading evaluation metric: accuracy

Defining Training Arguments with W&B Reporting...
Training on device: cuda:0
Reporting to W&B project: llm-cookbook-finetuning-demo


/tmp/ipykernel_31/1555914802.py:71: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Training finished.
***** train metrics *****
  epoch                    =        1.0
  total_flos               =  2774981GF
  train_loss               =     0.2577
  train_runtime            = 0:10:50.28
  train_samples_per_second =       34.6
  train_steps_per_second   =      1.083
Model and training state saved to ./fine_tune_classifier_wandb_output

Evaluating on the test set (logging to W&B)...


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


***** test metrics *****
  epoch                   =        1.0
  eval_accuracy           =     0.9231
  eval_loss               =     0.1974
  eval_runtime            = 0:03:57.43
  eval_samples_per_second =    105.291
  eval_steps_per_second   =      3.294
Test Set Evaluation Results:
{'eval_loss': 0.19743402302265167, 'eval_accuracy': 0.92308, 'eval_runtime': 237.4365, 'eval_samples_per_second': 105.291, 'eval_steps_per_second': 3.294, 'epoch': 1.0}


eval/accuracy,▁█
eval/loss,█▁
eval/runtime,▁█
eval/samples_per_second,▁█
eval/steps_per_second,█▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇████
train/global_step,▁▂▂▃▃▄▄▅▅▆▆▇▇████
train/grad_norm,▇▆▄█▂▁█▂▃▆▁▄▄▆
train/learning_rate,█▇▇▆▆▅▅▄▄▃▃▂▂▁
train/loss,█▃▂▂▂▁▁▂▁▁▁▂▂▁
eval/accuracy,0.92308



W&B run finished.


In [ ]:
# --- Recipe: Tracking Your Cook (Logging with W&B) ---
# Goal: Integrate Weights & Biases (W&B) for experiment tracking during fine-tuning.
# Note: Requires a W&B account (free for personal use).
#       Run `pip install wandb`
#       Log in via CLI: `wandb login` (or set WANDB_API_KEY environment variable).

import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
import os # For setting environment variables (optional)

# --- W&B Configuration ---

# Set your W&B project name. Runs will be logged here.
WANDB_PROJECT_NAME = "llm-cookbook-finetuning-demo"
os.environ["WANDB_PROJECT"] = WANDB_PROJECT_NAME
os.environ["WANDB_API_KEY"] = "INPUT YOUR API KEY"
# Optional: Set WANDB_LOG_MODEL to 'checkpoint' to automatically log model checkpoints to W&B
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

import wandb
wandb.login(key="INPUT YOUR API KEY")
wandb.init(project=WANDB_PROJECT_NAME, name=f"{MODEL_CHECKPOINT}-ft-{DATASET_NAME}-manual")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


##  Recipe: The Taste Test (Comparing Before & After Fine-Tuning)

In [ ]:
# --- Full Script: Evaluate Base → Fine-Tune → Evaluate Fine-Tuned Model on IMDB ---

import os
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

# --- Configuration ---
BASE_MODEL_CHECKPOINT = "distilbert-base-uncased"
FINE_TUNED_MODEL_PATH = "./fine_tune_classifier_output"
DATASET_NAME = "imdb"
BATCH_SIZE = 16
NUM_EPOCHS = 1
LEARNING_RATE = 2e-5

# --- Load Dataset ---
print("Loading dataset...")
raw_datasets = load_dataset(DATASET_NAME)
num_labels = raw_datasets["train"].features["label"].num_classes
test_dataset_raw = raw_datasets["test"]

# --- Load Tokenizer ---
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_CHECKPOINT)

# --- Load Metric ---
accuracy_metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy_metric.compute(predictions=predictions, references=labels)

# --- Tokenize Test Set (for both base and fine-tuned) ---
print("\nTokenizing test set...")
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_test_dataset = test_dataset_raw.map(tokenize_function, batched=True)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text"])
tokenized_test_dataset = tokenized_test_dataset.rename_column("label", "labels")
tokenized_test_dataset.set_format("torch")

# --- 1. Evaluate BASE Model ---
print(f"\n--- Evaluating BASE Model ({BASE_MODEL_CHECKPOINT}) ---")
try:
    base_model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_CHECKPOINT, num_labels=num_labels)
    print("Base model loaded.")

    eval_args = TrainingArguments(
        output_dir="./eval_base_output",
        per_device_eval_batch_size=BATCH_SIZE,
        do_train=False,
        do_eval=True,
        report_to="none",
        fp16=torch.cuda.is_available()
    )

    base_trainer = Trainer(
        model=base_model,
        args=eval_args,
        eval_dataset=tokenized_test_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics
    )

    base_eval_results = base_trainer.evaluate()
    base_accuracy = base_eval_results.get("eval_accuracy", "N/A")
    print("\nBase Model Evaluation Results:")
    print(base_eval_results)
except Exception as e:
    print(f"Error during base model evaluation: {e}")
    base_accuracy = "Error"

# --- 2. Fine-Tune the Model ---
print("\n--- Fine-tuning the model ---")

# Tokenize train/val
train_val_split = raw_datasets["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.remove_columns(["text"]).rename_column("label", "labels").with_format("torch")
tokenized_val = tokenized_val.remove_columns(["text"]).rename_column("label", "labels").with_format("torch")

# Load base model
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL_CHECKPOINT, num_labels=num_labels)

# Training arguments
train_args = TrainingArguments(
    output_dir=FINE_TUNED_MODEL_PATH,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model(FINE_TUNED_MODEL_PATH)
print(f"Fine-tuned model saved to {FINE_TUNED_MODEL_PATH}")

# --- 3. Evaluate FINE-TUNED Model ---
print(f"\n--- Evaluating FINE-TUNED Model ({FINE_TUNED_MODEL_PATH}) ---")
try:
    fine_tuned_model = AutoModelForSequenceClassification.from_pretrained(FINE_TUNED_MODEL_PATH)

    ft_trainer = Trainer(
        model=fine_tuned_model,
        args=eval_args,  # Reuse the same eval_args
        eval_dataset=tokenized_test_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer),
        compute_metrics=compute_metrics
    )

    ft_eval_results = ft_trainer.evaluate()
    ft_accuracy = ft_eval_results.get("eval_accuracy", "N/A")
    print("\nFine-Tuned Model Evaluation Results:")
    print(ft_eval_results)
except Exception as e:
    print(f"Error during fine-tuned model evaluation: {e}")
    ft_accuracy = "Error"

# --- 4. Final Comparison ---
print("\n--- Accuracy Comparison ---")
print(f"Base Model Accuracy:      {base_accuracy:.4f}" if isinstance(base_accuracy, float) else base_accuracy)
print(f"Fine-Tuned Model Accuracy:{ft_accuracy:.4f}" if isinstance(ft_accuracy, float) else ft_accuracy)


Loading dataset...
Loading tokenizer...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Tokenizing test set...

--- Evaluating BASE Model (distilbert-base-uncased) ---
Base model loaded.


/tmp/ipykernel_31/1973684533.py:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  base_trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Base Model Evaluation Results:
{'eval_loss': 0.6908515691757202, 'eval_accuracy': 0.54072, 'eval_runtime': 223.4219, 'eval_samples_per_second': 111.896, 'eval_steps_per_second': 3.5}

--- Fine-tuning the model ---


...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_31/1973684533.py:115: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.272000,0.215122,0.914400


Fine-tuned model saved to ./fine_tune_classifier_output

--- Evaluating FINE-TUNED Model (./fine_tune_classifier_output) ---


/tmp/ipykernel_31/1973684533.py:134: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  ft_trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



Fine-Tuned Model Evaluation Results:
{'eval_loss': 0.19901823997497559, 'eval_accuracy': 0.92188, 'eval_runtime': 222.6905, 'eval_samples_per_second': 112.263, 'eval_steps_per_second': 3.512}

--- Accuracy Comparison ---
Base Model Accuracy:      0.5407
Fine-Tuned Model Accuracy:0.9219
